In [1]:
#import pandas as pd
from bs4 import BeautifulSoup as bs
import re
from collections import defaultdict

In [2]:
with open("varia.xml", "r", encoding="utf-8") as file:
    # Read each line in the file, readlines() returns a list of lines
    content = file.readlines()
    # Combine the lines in the list into a string
    content = "".join(content)
    soup = bs(content, "xml")
with open("navneregister.xml", "r", encoding="utf-8") as file:
    # Read each line in the file, readlines() returns a list of lines
    content = file.readlines()
    # Combine the lines in the list into a string
    content = "".join(content)
    registry = bs(content, "xml")

In [3]:
d = defaultdict(dict)

docDates,regDates,mscontentscount,biblcount = 0,0,0,0
i=1

for item in soup.findAll("TEI"):
    itemDate = "N/A"
    itemType = item['subtype']
    if itemType == "dedication" or itemType == "greeting":
        xmlid = item['xml:id']
        #print(item["xml:id"])
        try:
            bibl = item.find("bibl")
            title = bibl.find("title")
            #print("Title in bibl element")
            biblcount += 1
        except:
            bibl = item.find("msContents")
            title = bibl.find("title")
            #print("Title in msContents element")
            mscontentscount += 1
        for match in title.findAll():
            match.unwrap()
        newTitle = ""
        for x in title.contents: # ??? kan ikke bruke title.contents[0]. Legger til i ny streng som workaround.
            newTitle+=" "+x
        title = newTitle.strip()
        title = re.sub(' +', ' ',title)
        title = re.sub('\n',"",title)
        itemDate = item.find("docDate")
        try:
            xmldate = itemDate['when']
            docDates+=1

        except:
            itemDate = item.find("date")
            xmldate = itemDate['when']
            regDates+=1
        date = itemDate['when']
        datenumbers = xmldate.replace("-","")
        
        if "Vudat" in xmlid:
            persid = xmlid.replace("Vudat","")
        else:
            persid = xmlid[1:]
        persid = persid.replace(datenumbers,"")
        if "-" in persid: # Indicates date range in xmlid.
            persid = persid[3:] # Remove -XX left behind.
        print("\n")
        d[xmlid]['title'] = title
        print(xmlid,title,itemType,date,"pe/orgID:",persid)
        d[xmlid]['type'] = itemType
        d[xmlid]['date'] = date
        d[xmlid]['ID'] = persid
        i+=1
print(i,"docDate elements:",docDates,"\nregular date elements:",regDates,"\nmscontentscount:",mscontentscount,"\nbibl count:",biblcount)



V18500415CLD Hilsen i Christopher L. Dues stambok greeting 1850-04-15 pe/orgID: CLD


V18560226NN Dedikasjon til ukjent mottaker i Gildet paa Solhoug dedication 1856-02-26 pe/orgID: NN


V1856HJ Dedikasjon til Herman Johnson i Gildet paa Solhoug dedication 1856 pe/orgID: HJ


V18571004MB Dedikasjon til Michael Birkeland i Fru Inger til Østeraad dedication 1857-10-04 pe/orgID: MB


V185712JCG Dedikasjon til J. Chr. Gamborg i Fru Inger til Østeraad dedication 1857-12 pe/orgID: JCG


V1858kongO2 Dedikasjon til prins Oscar (senere Oscar II) i Hærmændene paa Helgeland dedication 1858 pe/orgID: kongO2


V1858CRU Dedikasjon til Carl Richard Unger i Hærmændene paa Helgeland dedication 1858 pe/orgID: CRU


V1858KK Dedikasjon til Knud Knudsen i Hærmændene paa Helgeland dedication 1858 pe/orgID: KK


V1858CHo Dedikasjon til Chr. Holst i Hærmændene paa Helgeland dedication 1858 pe/orgID: CHo


V18600819ABla Dedikasjon til August Blanche i Hærmændene paa Helgeland dedication 1860-08-19 pe/orgID: 

In [4]:
iP=0
iO=0
found_persons = []
for item in d:
    flagFound = False
    whoid = d[item]['ID']
    persid = "pe"+whoid
    orgid = "org"+whoid
    try:
        persdiv = registry.find('div', {'xml:id':persid})
        name = persdiv.find('persName')
        for match in name.findAll():
            match.unwrap()
        newname = ""
        for x in name.contents: # ??? kan ikke bruke title.contents[0]. Legger til i ny streng som workaround.
            newname+=" "+x
        name = newname.strip()
        name = re.sub(' +', ' ',name)
        print("PERS - Title:",d[item]['title'],"To:",d[item]['ID'],"Registry:",name)
        iP+=1
        found_persons.append(item)
    except:
        try:
            orgid = registry.find('div', {'xml:id':orgid})
            name = orgid.find('item',{'rend':'name'})
            for match in name.findAll():
                match.unwrap()
            newname = ""
            for x in name.contents: # ??? kan ikke bruke title.contents[0]. Legger til i ny streng som workaround.
                newname+=" "+x
            name = newname.strip()
            name = re.sub(' +', ' ',name)
            print("ORG - Title:",d[item]['title'],"To:",d[item]['ID'],"Registry:",name)
            iO+=1
            found_persons.append(item)
        except:
            print(whoid)
print(iP,"personal names\n",iO,"organizations")

PERS - Title: Hilsen i Christopher L. Dues stambok To: CLD Registry: CHRISTOPHER [LORENTZ] DUE
NN
HJ
PERS - Title: Dedikasjon til Michael Birkeland i Fru Inger til Østeraad To: MB Registry: MICHAEL BIRKELAND
PERS - Title: Dedikasjon til J. Chr. Gamborg i Fru Inger til Østeraad To: JCG Registry: JOACHIM [CHRISTIAN] GAMBORG
kongO2
PERS - Title: Dedikasjon til Carl Richard Unger i Hærmændene paa Helgeland To: CRU Registry: CARL RICHARD UNGER
PERS - Title: Dedikasjon til Knud Knudsen i Hærmændene paa Helgeland To: KK Registry: KITTY (Christine Lange) KIELLAND
PERS - Title: Dedikasjon til Chr. Holst i Hærmændene paa Helgeland To: CHo Registry: CHRISTIAN HOLST
PERS - Title: Dedikasjon til August Blanche i Hærmændene paa Helgeland To: ABla Registry: AUGUST [THEODOR] BLANCHE
Skand
EZo
Skand
Skand
PERS - Title: Dedikasjon til Wilhelmine Holmboe (Schenstrøm) i Kjærlighedens Komedie To: WHo Registry: WILHELMINE HOLMBOE (Wielma Lhombino) SCHENSTRØM
HK
PERS - Title: Dedikasjon til Bernhard Dunker i

HF
Thea
PERS - Title: Hilsen i Dagny Bjørnsons autografbok To: DB Registry: DAGNY BJØRNSON
PERS - Title: Hilsen til Oda Nielsen (f. Larssen) på fotografi To: ONie Registry: ODA [LAURENZE HELMINE] f. Larsen NIELSEN
EG
PERS - Title: Hilsen til Wilhelmine Holmboe-Schenstrøm på Werenskiolds Ibsenportrett fra 1895 To: WHo Registry: WILHELMINE HOLMBOE (Wielma Lhombino) SCHENSTRØM
PERS - Title: Dedikasjon til Johan Herman Thoresen i John Gabriel Borkman To: JHT Registry: JOHAN HERMAN THORESEN
KFri
NN
PERS - Title: Dedikasjon til A. M. Lugné-Poë i John Gabriel Borkman To: AMLP Registry: AURÉLIEN [FRANÇOIS] MARIE LUGNÉ-POË
AMar
EAr
PERS - Title: Dedikasjon til Sigurd Ibsen i billedbibel To: SiI Registry: SIGURD IBSEN
PERS - Title: Inskripsjon på boks med penneholder (til pennesplitt), gitt i julegave til Aksel Arstal To: AArs Registry: AKSEL [KRISTIAN ANDERSEN] ARSTAL
IM
PERS - Title: Hilsen til Anna Stousland på fotografi To: ASto Registry: ANNA [JACOBINE ROSINE] STOUSLAND
PERS - Title: Hilsen

In [5]:
losttitles = []
for item in d:
    if item not in found_persons and d[item]['ID'] != "NN":
        print(item,"not found. Title",d[item]['title'],"ID:",d[item]['ID'])
        losttitles.append(d[item]['title'])

V1856HJ not found. Title Dedikasjon til Herman Johnson i Gildet paa Solhoug ID: HJ
V1858kongO2 not found. Title Dedikasjon til prins Oscar (senere Oscar II) i Hærmændene paa Helgeland ID: kongO2
V1860Skand not found. Title Dedikasjon til Den skandinaviske Forening i Rom i Nytaarsgave for Illustreret Nyhedsblads Abonnenter av Paul Botten Hansen ID: Skand
V18610410EZo not found. Title Dedikasjon til Emilie Zogbaum i et Ibsen-verk ID: EZo
V1861Skand not found. Title Dedikasjon til Den skandinaviske Forening i Rom i Nytaarsgave for Illustreret Nyhedsblads Abonnenter av Paul Botten Hansen ID: Skand
V1862Skand not found. Title Dedikasjon til Den skandinaviske Forening i Rom i Nytaarsgave for Illustreret Nyhedsblads Abonnenter av Paul Botten Hansen ID: Skand
V1862HK not found. Title Dedikasjon til Henriette Konow i Kjærlighedens Komedie ID: HK
V18640616NiFi not found. Title Dedikasjon til Niels Fischer i Kongs-Emnerne ID: NiFi
V1864JPLN not found. Title Dedikasjon til J. Peter L. Nielsen i Ko

In [6]:
i=0
orgs = []
pers = []
for orgid in registry.findAll('div', {'xml:id':re.compile('org.*')}):
    name = orgid.find('item',{'rend':'name'})
    for match in name.findAll():
        match.unwrap()
    newname = ""
    for x in name.contents: # ??? kan ikke bruke title.contents[0]. Legger til i ny streng som workaround.
        newname+=" "+x
    name = newname.strip()
    name = re.sub(' +', ' ',name)
    print(name)
    i+=1
    orgs.append(name)
print(i,"organizations")
i=0
for persid in registry.findAll('div', {'xml:id':re.compile('pe.*')}):
    name = persid.find('item',{'rend':'name'})
    for match in name.findAll():
        match.unwrap()
    newname = ""
    for x in name.contents: # ??? kan ikke bruke title.contents[0]. Legger til i ny streng som workaround.
        newname+=" "+x
    name = newname
    name = re.sub(' +', ' ',name)
    print(name)
    i+=1
    pers.append(name)
print(i,"persons")

GEORG 2.
SILLE (Cecilie Marie) f. Gade JØRGENSEN
AFTENBLADET
AFTENPOSTEN
BANCO NAPOLETANA
BERGENS NATIONALE SCENE
BERGENS THEATERFORENING
BERLINGSKE TIDENDE
BURGTHEATER
CHRISTIANIA BYFOGEDCONTOR
CHRISTIANIA THEATER
CHRISTIANIA-POSTEN
DAGBLADET
DAGS-TELEGRAPHEN
DEN FRISINNEDE STUDENTERFORENING
DEN GYLDENDALSKE BOGHANDEL
DEN NORSKE FORFATTERFORENING
DEN SKANDINAVISKE FORENING I ROMA
DET AKADEMISKE KOLLEGIUM
DET BRITISK-SKANDINAVISKE SELSKAB
DET KONGELIGE NORSKE VIDENSKABERS SELSKAB
DET KONGELIGE THEATER I KØBENHAVN
DET NORSKE STUDENTERSAMFUND
DET NORSKE THEATER I BERGEN
DEUTSCHE GENOSSENSCHAFT DRAMATISCHER AUTOREN UND COMPONISTEN
DEUTSCHES VOLKSTHEATER
DIREKSJONEN FOR DAMPSKIPET DRONNINGEN
DRESDNER BEZIRKS-GERICHTS-AMT
FELIX BLOCH ERBEN
FINANTS- OG TOLDDEPARTEMENTET (FINANSDEPARTEMENTET)
GIORDANO BRUNO-KOMITEEN
HOHLENBERG
HOTEL GRÖBNER
ILLUSTRERET NYHEDSBLAD
JENSEN CO.
KIRKE- OG UNDERVISNINGSDEPARTEMENTET, eller: DEPARTEMENTET FOR KIRKE- OG UNDERVISNINGSVESENET, gjerne forkortet: KIRKEDE

 FREDERIK HERMAN (Friedrich Hermann) WOLFHAGEN
 CARSTEN [LÜTKEN] WOLL
 LOUIS FRANÇOIS MICHEL RAYMOND WOLOWSKI
 CHARLOTTE WOLTER
 ALFRED [FREIHERR] VON WOLZOGEN
 [JOHAN] JAKOB WOXEN
 KARL (Carl Gustav Marius) WOXEN
 BERNHARD WULFF
 [CARL] ERIC [ADAM] ZACHRISON
 RIKKA (Ulrikke Cathrine) f. og tidligere g. Schulerud ZEIER 

 herr ZIMMERMANN
 AUGUSTE (Gustava Sophia Agneta) f. Raddatz ZINCK
 ÉMILE ZOLA
 HEINRICH ZSCHALIG
 DIKKEN (Henrikke Barbara Wind Daae) ZWILGMEYER
 CLAES GUSTAF ÖSTBERG
 VILHELM (Wilhelm) ØSTERGAARD
 HANS ÖSTERLING
 GURLI (Johanna Carolina) (1883 g. Ulff) ÅBERG
 [CHRISTIAN] EBERHARD ÅBJÖRNSSON
 WILLIAM [GOTTLIEB] ÅBJÖRNSSON
 SVEA ÅHMAN
 HENRIK ÅKERMAN
 LOUISE JULIE JACQUETTE f. Liljencrantz ÅKERMAN
 IDA [EMILIA] AALBERG
 CARL SIGVARDTH AAMODT
 [HANSINE] HENRIETTE ANDERSEN f. Vogt AARS
 [JACOB] JONATHAN AARS
 IVAR [ANDREAS] AASEN
1634 persons


In [7]:
titlesstillmissing,nodontadd = [],[]
v=0
for x in losttitles:
    z = x.strip()
    #print("Mod 0:",z)
    z = z.replace("Dedikasjon til","")
    z = z.replace("Hilsen til","")
    z = z.replace("Hilsener til","")
    #print("Mod 1:",z)
    if " i " in z:
        a = z.split(" i ")
    elif " på " in z:
        a = z.split(" på ")
    elif " til " in z:
        a = z.split(" til ")
    elif " bakpå " in z:
        a = z.split(" bakpå ")
    else:
        a = [z]
        #print("Neither",x)
    if "Hilsen" in a[0]:
        z = a[1]
    else:
        z = a[0]
    #print(z)
    z = z.strip()
    #print("Mod 2:",z)
    for xx in orgs:
        if z.lower() in xx.lower():
            print("Mod 3O:",x,"/",z,"/",xx)
            v+=1
            nodontadd.append(z)
    for xx in pers:
        if z.lower() in xx.lower():
            print("Mod 3P:",x,"/",z,"/",xx)
            v+=1
            nodontadd.append(z)
    if z not in nodontadd:
        titlesstillmissing.append(z)
    #print("")
print(v,"new entries")

Mod 3O: Dedikasjon til Den skandinaviske Forening i Rom i Nytaarsgave for Illustreret Nyhedsblads Abonnenter av Paul Botten Hansen / Den skandinaviske Forening / DEN SKANDINAVISKE FORENING I ROMA
Mod 3O: Dedikasjon til Den skandinaviske Forening i Rom i Nytaarsgave for Illustreret Nyhedsblads Abonnenter av Paul Botten Hansen / Den skandinaviske Forening / DEN SKANDINAVISKE FORENING I ROMA
Mod 3O: Dedikasjon til Den skandinaviske Forening i Rom i Nytaarsgave for Illustreret Nyhedsblads Abonnenter av Paul Botten Hansen / Den skandinaviske Forening / DEN SKANDINAVISKE FORENING I ROMA
Mod 3P: Hilsen til Møller på baksiden av fotografi fra Forum Romanum / Møller /  FRANTZ PECKEL MØLLER
Mod 3P: Hilsen til Møller på baksiden av fotografi fra Forum Romanum / Møller /  SOPHIE [MAREN] MØLLER
Mod 3P: Hilsen til Møller på baksiden av fotografi fra Forum Romanum / Møller /  HELENE [MØLLER] f. Linde SONTUM
Mod 3P: Hilsen til Møller på baksiden av fotografi fra Forum Romanum / Møller /  MARCUS [MØLLE

In [12]:
titlesstillmissing

['Herman Johnson',
 'prins Oscar (senere Oscar II)',
 'Emilie Zogbaum',
 'Henriette Konow',
 'Niels Fischer',
 'J. Peter L. Nielsen',
 'Niels A. Nicolaysen',
 'Innførsel',
 'gjestebok, La Rosa Magra',
 'Augusta Nordenfeldt',
 'J.C. Heuch',
 'Universitetsbiblioteket',
 'Clara Ziegler',
 'Louise Werner',
 'Karl Frenzel',
 'C. Möller',
 'Ludwig Diestel',
 'Rosamunde Königsdörffer',
 'Ludwig Diestel',
 'Robert Waldmüller',
 'Wilhelm von Wartenegg',
 'V. v. Maurer',
 'Wilhelm Henzen',
 'Ludwig Diestel',
 'Max Ring',
 'C. Abrahams på baksiden av fotografi limt inn',
 'Carl Stieler',
 'fru Clemence Nielsen',
 'M. Larsson',
 'familien Gröbners stambok',
 'Leopold von Sacher-Masochs album',
 'ukjent',
 'den svensk-norske understøttelsesforening',
 'Janet Achurch',
 'Clara Ebert',
 'Hedwig Grønvold',
 'Henrik Jæger',
 'Alice Berend',
 'Emilie Mataja/Emil Marriots visittkort',
 'Gerda Lundequist',
 'Sigrid Arnoldsons stambok',
 'Illa Christensen',
 'Granville Bantock',
 'Sitat fra En Folkefiende 

In [14]:
PossibleTokenMatchReg = []
LeftoverNames = []
                      
for entry in titlesstillmissing:
    string = re.sub('\(', '',(re.sub('\)', '', str(entry))))
    stringLower = string.lower()
    tokens = stringLower.split()
    TokenMatchReg = []            
    for person in pers:
        personLower = person.lower()
        personTokens = re.sub('\(', '',(re.sub('\)', '',(re.sub('\]', '',(re.sub('\[', '', personLower)))))))
    # personTokens = personTokens.split()
        
        if all(x in personTokens for x in tokens):
            TokenMatchReg.append( entry+ '/' + personTokens)
        
        
    if len(TokenMatchReg) != 0:
        PossibleTokenMatchReg.append(TokenMatchReg)
    else:
            LeftoverNames.append(entry)
        
print(PossibleTokenMatchReg)
print(LeftoverNames)
print(len(LeftoverNames))
            

[['Hedwig Grønvold/ hedwig f. hermann grønvold'], ['Henrik Jæger/ hans henrik jæger', 'Henrik Jæger/ henrik bernhard jæger']]
['Herman Johnson', 'prins Oscar (senere Oscar II)', 'Emilie Zogbaum', 'Henriette Konow', 'Niels Fischer', 'J. Peter L. Nielsen', 'Niels A. Nicolaysen', 'Innførsel', 'gjestebok, La Rosa Magra', 'Augusta Nordenfeldt', 'J.C. Heuch', 'Universitetsbiblioteket', 'Clara Ziegler', 'Louise Werner', 'Karl Frenzel', 'C. Möller', 'Ludwig Diestel', 'Rosamunde Königsdörffer', 'Ludwig Diestel', 'Robert Waldmüller', 'Wilhelm von Wartenegg', 'V. v. Maurer', 'Wilhelm Henzen', 'Ludwig Diestel', 'Max Ring', 'C. Abrahams på baksiden av fotografi limt inn', 'Carl Stieler', 'fru Clemence Nielsen', 'M. Larsson', 'familien Gröbners stambok', 'Leopold von Sacher-Masochs album', 'ukjent', 'den svensk-norske understøttelsesforening', 'Janet Achurch', 'Clara Ebert', 'Alice Berend', 'Emilie Mataja/Emil Marriots visittkort', 'Gerda Lundequist', 'Sigrid Arnoldsons stambok', 'Illa Christensen',